In [1]:
import csv
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
import tabula
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter

In [2]:
lista_tabelas = tabula.read_pdf('Territorios_Rurais.pdf', pages='all', lattice=True)

In [3]:
len(lista_tabelas)

9

In [4]:
lista_tabelas[6]

,Do Distrito,Do concelho,As Freguesias
0,NaN,Lousada,Todas as freguesias
1,NaN,Paços de Ferreira,Todas as freguesias
2,NaN,Vila do Conde,"Árvore, Aveleda, Azurara, Fajozes, Gião, Guilh..."
3,Santarém,Alcanena,Todas as freguesias
4,NaN,Ferreira do Zêzere,Todas as freguesias
5,NaN,Ourém,Todas as freguesias
6,NaN,Rio Maior,Todas as freguesias
7,NaN,Tomar,"Asseiceira, Carregueiros, Olalhas, Paialvo, Sã..."
8,NaN,Torres Novas,Todas as freguesias
9,NaN,Mação,Todas as freguesias


In [5]:
for j in range(len(lista_tabelas)):
    linhas = lista_tabelas[j][lista_tabelas[j]['As Freguesias'].isnull()].index
    linhas = list(reversed(linhas))
    for i in linhas:
        valores_concelho = lista_tabelas[j]['Do Distrito'][i:i+1]
        lista_tabelas[j].iloc[i:i+1, 2:] = lista_tabelas[j].iloc[i:i+1, 1:-1].values
        lista_tabelas[j].iloc[i:i+1, 1] = valores_concelho
        lista_tabelas[j].loc[i, 'Do Distrito'] = None

In [6]:
# Concatenar todas as tabelas em um único DataFrame
df_final = pd.concat(lista_tabelas, ignore_index=True)

In [7]:
# Inicializar a variável para armazenar o último valor não nulo da coluna "Do Distrito"
ultimo_valor_distrito = None

# Iterar sobre o DataFrame
for index, row in df_final.iterrows():
    # Verificar se o valor da coluna "Do Distrito" é nulo
    if pd.isnull(row['Do Distrito']):
        # Substituir o valor nulo pelo último valor não nulo encontrado
        df_final.at[index, 'Do Distrito'] = ultimo_valor_distrito
    else:
        # Atualizar o último valor não nulo encontrado
        ultimo_valor_distrito = row['Do Distrito']

df_final = df_final.applymap(lambda x: x.replace('\r', ' '))

In [8]:
indices_com_virgula = df_final[df_final['As Freguesias'].str.contains(',')].index
linhas = list(reversed(indices_com_virgula))
for i in linhas:
    freguesias_linha = df_final.loc[i, 'As Freguesias']
    freguesias = freguesias_linha.split(',')

    coluna_anterior1 = df_final.iloc[i, 0]
    coluna_anterior2 = df_final.iloc[i, 1]

    # Remover espaços extras no início e no final de cada freguesia
    freguesias = [f.strip() for f in freguesias]
    
    df_temp = pd.DataFrame({'Do Distrito': [coluna_anterior1] * len(freguesias),
                            'Do concelho': [coluna_anterior2] * len(freguesias),
                            'As Freguesias': freguesias})

    df_final = df_final.drop(i).reset_index(drop=True)
    df_final = pd.concat([df_final, df_temp], ignore_index=True)

In [9]:
df_final = df_final.sort_values(by='Do Distrito').reset_index(drop=True)
df_final.iloc[592]

Do Distrito                  Setúbal
Do concelho                 Grândola
As Freguesias    Todas as freguesias
Name: 592, dtype: object

In [10]:
# Salvar o DataFrame em um arquivo CSV
df_final.to_csv('ZonasRuraisRaw.csv', index=False, header=True)

- - -